<h1> Aplicação do SVD ao dataset em português do Word2Vec </h1>
<h3> O objetivo desse trabalho é compreender como o a técnica de Word2Vec funciona e utilizar o modelo de SVD para reduzir as dimensões do dataset de Word2Vec disponibilizado pelo Laboratório de Computação e Linguística da USP.</h3>
<h4><ol>
    <li>Introdução</li>
    <li>Objetivo</li>
    <li>Metodologia</li>
    <li>Estudo de Caso</li>
    <li>Conclusões</li>
    <li>Referências</li>
</ol></h4>

<h4>1. Introdução</h4>

<p>Como definir que uma frase ou uma palavra é similar a outra? Para resolver esse problema uma das soluções utilizadas é o treinamento de uma Rede Neural que usa como base uma representação vetorial das palavras. Isso pode ser observado nesse artigo bem explicativo do Luiza Labs [1].</p>

<p>Contudo, esse dataset de vetores de palavras (Word2Vec) em geral tem uma alta dimensionalidade, mais de 300 dimensões. Nesse trabalho deseja-se utilizar o método do SVD [2], para reduzir essa dimensionalidade preservando os "agrupamentos" encontrados após o treinamento.</p>


<h4>2. Objetivo</h4>

<p> Com o uso do SVD, reduzir o dataset de 300 para 100, 50, 10 e 2 dimensões. Verificar se ainda são preservadas as principais características, em especial algumas operações conhecidas como:</p>

<p><i>Rei - Homem + Mulher = Rainha</i></p>

<h4>3. Metodologia</h4>

<ul>
    <li><p>Usar o código de SVD do Numpy [3] </p></li>
    <li><p>Uso do Modelo Pré-Treinado de Word2Vec [2] </p></li>
    <li><p>Aplicação de Visualização [4] </p></li>

</ul>

<h5>3.1 Representação Vetorial</h5>

<p> Sem dúvidas uma das melhores referências para entendimento da representação vetorial, [2].


<h4>4. Estudo de Caso</h4>

<p>Uso do Modelo Pré-Treinado de Word2Vec para português disponibilizado e treinado utilizando o algoritmo Skip-Gram, inicialmente foi tentado utilizar o modelo com 300 dimensões, contudo por problemas de memória, não foi possível fazer load de toda a matriz, logo diminuimos o escopo para uma matriz com 300 dimensoes e 20k linhas</p>

<p>Verificamos inicialmente quais os indices das palavras a serem testadas para posteriormente fazer esse corte. </p>



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gensim

In [35]:
# import data
import os
dirname = os.path.realpath('..')

def model_trigram():
    filename = dirname+'/SVD2Word2Vec/word2vec_copy/exemplo/wiki.pt.trigram.vector'
    model = gensim.models.KeyedVectors.load_word2vec_format(filename,binary=True)
    model.init_sims(replace=False)
    #model.init_sims(replace=True) # to reduce memory usage
    return model
    
def model_skipgram_usp():
    filename = dirname+'/SVD2Word2Vec/word2vec_copy/exemplo/skip_s300.txt'
    model = gensim.models.KeyedVectors.load_word2vec_format(filename,binary=False)
    model.init_sims(replace=False)
    #model.init_sims(replace=True) # to reduce memory usage
    return model

def model_skipgram_usp_50():
    filename = dirname+'/SVD2Word2Vec/word2vec_copy/exemplo/skip_s50.txt'
    model = gensim.models.KeyedVectors.load_word2vec_format(filename,binary=False)
    model.init_sims(replace=False)
    #model.init_sims(replace=True) # to reduce memory usage
    return model

def compare_Woman_King_Man(model):
    result = model.most_similar(positive=['mulher', 'rei'], negative=['homem'])
    print("\n{}: {:.4f}\n".format(*result[0]))

# load model
model = model_skipgram_usp()
model_50 = model_skipgram_usp_50()

In [3]:
def most_similar(vec1,matrix):
    
    index_list = list()
    cossine_distance_list = list()
    
    cossine_distance = np.dot(vec1,matrix[0])/(np.linalg.norm(vec1)*np.linalg.norm(matrix[0]))
    
    new_matrix = list()
    
    for i in range(len(matrix)):
        cossine_distance_loop = np.dot(vec1,matrix[i])/(np.linalg.norm(vec1)*np.linalg.norm(matrix[i]))
        new_matrix.append(cossine_distance_loop)

    index_index = sorted(range(len(new_matrix)), key=lambda k: new_matrix[k])
    new_cossine_distance_list = list()
    new_index_list = list()
        
    for i in index_index:
        new_cossine_distance_list.append(new_matrix[i])
        
    index_index = index_index[len(index_index)-10:]
    cossine_distance_list = new_cossine_distance_list[len(new_cossine_distance_list)-10:]
        
    return index_index,cossine_distance_list


In [4]:
#model.wv.syn0  # Input Embedding Matrix
#model.syn1     # Output embedding //with hierarchical softmax (hs=1)//
#model.syn1neg  # Output embedding //when it uses negative sampling (negative>0)//

np_matrix = np.asarray(model.wv.syn0)
print('\n Shape of Dataset: {}'.format(np_matrix.shape))

# Reduce number of lines in order to avoid memory problems;
np_matrix_reduced = np_matrix[:20000]
print('\n Shape of Dataset (reduced): {}'.format(np_matrix_reduced.shape))

U, S, Vt = np.linalg.svd(np_matrix_reduced)

print('\n  -Shape of Matrix U: {}'.format(U.shape))
print('\n  -Shape of Matrix S: {}'.format(S.shape))
print('\n  -Shape of Matrix Vt: {}'.format(Vt.shape))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """



 Shape of Dataset: (929606, 300)

 Shape of Dataset (reduced): (20000, 300)

  -Shape of Matrix U: (20000, 20000)

  -Shape of Matrix S: (300,)

  -Shape of Matrix Vt: (300, 300)


In [30]:
print("\nMatrix Reconstruction: \n")

matrix_reconstructed_2_dim = np.matrix(U[:, :2]) * np.diag(S[:2]) * np.matrix(Vt[:2, :])
matrix_reconstructed_10_dim = np.matrix(U[:, :10]) * np.diag(S[:10]) * np.matrix(Vt[:10, :])
matrix_reconstructed_50_dim = np.matrix(U[:, :50]) * np.diag(S[:50]) * np.matrix(Vt[:50, :])
matrix_reconstructed_100_dim = np.matrix(U[:, :100]) * np.diag(S[:100]) * np.matrix(Vt[:100, :])



# get the index of the words
woman_index = model.wv.vocab['mulher'].index
king_index = model.wv.vocab['rei'].index
man_index = model.wv.vocab['homem'].index
queen_index = model.wv.vocab['rainha'].index

print("\nWoman Index: {}".format(woman_index))
print("King Index: {}".format(king_index))
print("Man Index: {}".format(man_index))
print("Queen Index: {}\n".format(queen_index))



Matrix Reconstruction: 


Woman Index: 445
King Index: 666
Man Index: 294
Queen Index: 1989



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


<p>Obs. podemos verificar que a metodologia está correta com a implementada pelo Gensim Word2Vec<p>

In [31]:
def modify_matrix(matrix):
    new_matrix = list()

    for i in matrix:
        a = i
        a = np.transpose(a)
        a = np.squeeze(np.asarray(a))
        new_matrix.append(a)

    new_matrix = np.asarray(new_matrix)
    return new_matrix

In [34]:
# now compute King - Man + Woman

# 1. Find the most similar (result from Gensim model)
print("\n\n ##### (result from Gensim model) ######")
compare_Woman_King_Man(model)

# 2. Find the most similar (result direct from the matrix of the trainned model)
# obs: We want to find the result result of 1, just to see if the method is correct;
woman_vector = np_matrix[woman_index]
king_vector = np_matrix[king_index]
man_vector = np_matrix[man_index]

vec1 = king_vector + woman_vector - man_vector
index_index,similarity_list = most_similar(vec1,np_matrix)


print("\n\n ##### (result from new Matrix 300 dim) ######")
k = 0
for i in index_index:
    print("{}: {:.4f}, index: {}".format(model.wv.index2word[i],similarity_list[k],i))
    k += 1

print("\n\n")
result = model.most_similar(np.array([vec1]))
for i in result:
    print(i)


print("\n\n\nOBSERVA-SE AQUI QUE A METODOLOGIA DE CALCULO ESTÁ CORRETA, O MESMO RESULTADO PARA O MODELO GENSIM TREINADO E O MODELO UTILIZANDO DIRETAMENTE A MATRIX DE PESOS EXTRAÍDA DO MODELO\n\n\n")
    
    
# 3. Find the most similar (result from new Matrix 100 dim)
matrix_reconstructed_100_dim = modify_matrix(matrix_reconstructed_100_dim)

woman_vector = matrix_reconstructed_100_dim[woman_index]
king_vector = matrix_reconstructed_100_dim[king_index]
man_vector = matrix_reconstructed_100_dim[man_index]

vec1 = king_vector - man_vector + woman_vector
index_index,similarity_list = most_similar(vec1,matrix_reconstructed_100_dim)


print("\n\n ##### (result from new Matrix 100 dim) ######")
k = 0
for i in index_index:
    print("{}: {:.4f}, index: {}".format(model.wv.index2word[i],similarity_list[k],i))
    k += 1



# 4. Find the most similar (result from new Matrix 50 dim)
matrix_reconstructed_50_dim = modify_matrix(matrix_reconstructed_50_dim)

woman_vector = matrix_reconstructed_50_dim[woman_index]
king_vector = matrix_reconstructed_50_dim[king_index]
man_vector = matrix_reconstructed_50_dim[man_index]

vec1 = king_vector - man_vector + woman_vector
index_index,similarity_list = most_similar(vec1,matrix_reconstructed_50_dim)

print("\n\n ##### (result from new Matrix 50 dim) ######")
k = 0
for i in index_index:
    print("{}: {:.4f}, index: {}".format(model.wv.index2word[i],similarity_list[k],i))
    k += 1


# 5. Find the most similar (result from new Matrix 10 dim)
new_matrix_reconstructed_10_dim = modify_matrix(matrix_reconstructed_10_dim)

woman_vector = new_matrix_reconstructed_10_dim[woman_index]
king_vector = new_matrix_reconstructed_10_dim[king_index]
man_vector = new_matrix_reconstructed_10_dim[man_index]

vec1 = king_vector - man_vector + woman_vector
index_index,similarity_list = most_similar(vec1,new_matrix_reconstructed_10_dim)


print("\n\n ##### (result from new Matrix 10 dim) ######")
k = 0

for i in index_index:
    print("{}: {:.4f}, index: {}".format(model.wv.index2word[i],similarity_list[k],i))
    k += 1

# 6. Find the most similar (result from new Matrix 2 dim)
matrix_reconstructed_2_dim = modify_matrix(matrix_reconstructed_2_dim)

woman_vector = matrix_reconstructed_2_dim[woman_index]
king_vector = matrix_reconstructed_2_dim[king_index]
man_vector = matrix_reconstructed_2_dim[man_index]

vec1 = king_vector - man_vector + woman_vector
index_index,similarity_list = most_similar(vec1,matrix_reconstructed_2_dim)


print("\n\n ##### (result from new Matrix 2 dim) ######")
k = 0
for i in index_index:
    print("{}: {:.4f}, index: {}".format(model.wv.index2word[i],similarity_list[k],i))
    k += 1
    




 ##### (result from Gensim model) ######

rainha: 0.6601



 ##### (result from new Matrix 300 dim) ######
ah-hotep: 0.6332, index: 285429
esposa: 0.6358, index: 1278
príncipe-herdeiro: 0.6372, index: 53146
sobrinha: 0.6386, index: 11072
princesa: 0.6451, index: 2872
rainha-viúva: 0.6487, index: 115749
primogénita: 0.6598, index: 122910
consorte: 0.6866, index: 17430
rainha: 0.6870, index: 1989
rei: 0.7265, index: 666



('rei', 0.7265248894691467)
('rainha', 0.6869875192642212)
('consorte', 0.6865828037261963)
('primogénita', 0.6597862243652344)
('rainha-viúva', 0.6487346887588501)
('princesa', 0.645134449005127)
('sobrinha', 0.6385774612426758)
('príncipe-herdeiro', 0.6372197270393372)
('esposa', 0.6358453631401062)
('ah-hotep', 0.6331871747970581)
OBSERVA-SE AQUI QUE A METODOLOGIA DE CALCULO ESTÁ CORRETA, O MESMO RESULTADO PARA O MODELO GENSIM TREINADO E O MODELO UTILIZANDO DIRETAMENTE A MATRIX DE PESOS EXTRAÍDA DO MODELO


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).




 ##### (result from new Matrix 100 dim) ######
regente: 0.7210, index: 9014
esposa: 0.7271, index: 1278
neta: 0.7301, index: 10943
rei: 0.7396, index: 666
imperatriz: 0.7477, index: 8680
filha: 0.7499, index: 784
sobrinha: 0.7635, index: 11072
princesa: 0.7817, index: 2872
rainha: 0.7956, index: 1989
consorte: 0.7965, index: 17430


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).




 ##### (result from new Matrix 50 dim) ######
cunhada: 0.7845, index: 17450
imperatriz: 0.7904, index: 8680
esposa: 0.7906, index: 1278
neta: 0.7923, index: 10943
filha: 0.7983, index: 784
viúva: 0.8001, index: 6150
sobrinha: 0.8137, index: 11072
princesa: 0.8378, index: 2872
rainha: 0.8398, index: 1989
consorte: 0.8439, index: 17430


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).




 ##### (result from new Matrix 10 dim) ######
duquesa: 0.9115, index: 10908
espírita: 0.9166, index: 15683
herdeira: 0.9178, index: 15055
regência: 0.9195, index: 12420
sofia: 0.9204, index: 6988
honra: 0.9242, index: 2442
primeira-dama: 0.9262, index: 19593
rainha: 0.9326, index: 1989
imperatriz: 0.9338, index: 8680
princesa: 0.9374, index: 2872


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:83: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).




 ##### (result from new Matrix 2 dim) ######
regente: 0.7210, index: 9014
esposa: 0.7271, index: 1278
neta: 0.7301, index: 10943
rei: 0.7396, index: 666
imperatriz: 0.7477, index: 8680
filha: 0.7499, index: 784
sobrinha: 0.7635, index: 11072
princesa: 0.7817, index: 2872
rainha: 0.7956, index: 1989
consorte: 0.7965, index: 17430


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:100: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


<hr>
<p> Agora vamos comparar o resultado para 50 dimensoes usando o SVD e para um modelo pre-treinado em 50 dimensões </p>
<hr>

In [37]:
#model.wv.syn0  # Input Embedding Matrix
#model.syn1     # Output embedding //with hierarchical softmax (hs=1)//
#model.syn1neg  # Output embedding //when it uses negative sampling (negative>0)//

np_matrix = np.asarray(model_50.wv.syn0)
print('\n Shape of Dataset: {}'.format(np_matrix.shape))

# Reduce number of lines in order to avoid memory problems;
np_matrix_reduced = np_matrix[:20000]
print('\n Shape of Dataset (reduced): {}'.format(np_matrix_reduced.shape))



 Shape of Dataset: (929606, 50)

 Shape of Dataset (reduced): (20000, 50)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """


In [44]:
# 1. Find the most similar (result direct from the matrix of the trainned model 50 dim)
woman_vector = np_matrix[woman_index]
king_vector = np_matrix[king_index]
man_vector = np_matrix[man_index]

vec1 = king_vector - man_vector + woman_vector
index_index,similarity_list = most_similar(vec1,np_matrix)

print("\n\n ##### (result from new Matrix 50 dim) ######")
k = 0
for i in index_index:
    print("{}: {:.4f}, index: {}".format(model_50.wv.index2word[i],similarity_list[k],i))
    k += 1
    
# 2. Find the most similar (result from Matrix 50 dim AFTER SVD on the 300 dim matrix)
woman_vector = matrix_reconstructed_50_dim[woman_index]
king_vector = matrix_reconstructed_50_dim[king_index]
man_vector = matrix_reconstructed_50_dim[man_index]

vec1 = king_vector - man_vector + woman_vector
index_index,similarity_list = most_similar(vec1,matrix_reconstructed_50_dim)

print("\n\n ##### (result from new Matrix 50 dim SVD) ######")
k = 0
for i in index_index:
    print("{}: {:.4f}, index: {}".format(model_50.wv.index2word[i],similarity_list[k],i))
    k += 1




 ##### (result from new Matrix 50 dim) ######
consorte: 0.8546, index: 17430
imperatriz-mãe: 0.8548, index: 158221
imperatriz: 0.8556, index: 8680
infanta: 0.8662, index: 22059
esposa: 0.8672, index: 1278
rainha-consorte: 0.8763, index: 119481
rainha-viúva: 0.8763, index: 115594
rainha-mãe: 0.8869, index: 57958
rainha: 0.8892, index: 1989
princesa: 0.8987, index: 2872


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':




 ##### (result from new Matrix 50 dim SVD) ######
vínculos: 0.7845, index: 17450
imperatriz: 0.7904, index: 8680
esposa: 0.7906, index: 1278
neta: 0.7923, index: 10943
filha: 0.7983, index: 784
viúva: 0.8001, index: 6150
sobrinha: 0.8137, index: 11072
princesa: 0.8378, index: 2872
rainha: 0.8398, index: 1989
consorte: 0.8439, index: 17430


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


<h4>5. Conclusões</h4>

<p> A ideia inicial era realizar o comparativo de diversas operações do Word2Vec, contudo, muito do tempo foi gasto debugando o código para que funcionasse com a quantidade de memória disponível. Tentei inclusive usar uma máquina virtual da IBM Cloud para carregar totalmente mas sem sucesso.</p>

<p> Além disso, grande parte do trabalho de leitura para entendimento do problema, como de fato funciona o modelo Word2Vec para assim então aplicar o método do SVD </p>

<p> Por fim podemos ver que a redução de dimensionalidade preserva algumas das características originais, no caso de 10 dimensões temos ainda resultados interessantes, contudo, para 2 dimensões, os resultados são bem ruins. Outro fato interessante é que a similaridade, medida pelo cosseno entre os vetores, fica cada vez mais alta, ainda que a similaridade real não seja perfeita. É interessante notar que isso acontece por que quando diminuindo dimensões estamos "achatando" os vetores, assim, de fato, eles cada vez mais vão estar mais próximos, de maneira geral. </p>

<p> Foi comparado ao final um modelo treinado especificamente para 50 dimensoes, ou seja no treinamento da Rede Neural que originou essa matriz, tinham 50 neuronios na camada intermediária. E foi comparado o resultado com uma matriz reduzida pelo SVD, de 300 para 50 dimensões. Os resultados ficaram bem próximos, ainda que a vantagem esteja com a rede treinada diretamente para 50 dimensões.</p>

<p> Uma comparação a ser feita que pode ser interessante é realizar o treinamento novamente do modelo Word2Vec só que dessa vez já para a dimensão reduzida, ou seja, treinar para 2, 10 features. E comparar os resultados da Rede Neural treinada já na dimensão inferior com os resultados do SVD aplicado em uma matriz com dimensão superior, como no caso desse trabalho. </p>


<p> Nesse repositório ainda encontra-se uma aplicação para visualização dessas palavras próximas e das operações com vetores, contudo, não foi possível inputar no modelo Gensim (que é a base para essas operações na aplicação Word2Vec-pt-br) as novas matrizes calculadas após o SVD. Seria necessário mais tempo de pesquisa para descobrir como fazer essa alteração. Ainda assim, como teve-se que reduzir o número de linhas utilizadas na matriz, teríamos problema de compatibilidade. <p>
    
<p> Mas a ideia era ter uma representação visual do que está acontecendo com os vetores ao fazer a redução de dimensão. </p>

<h4>6. Referências</h4>

[1] https://medium.com/luizalabs/similaridade-entre-t%C3%ADtulos-de-produtos-com-word2vec-5e26199862f0

[2] http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

[3] https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.linalg.svd.html

[4] https://github.com/felipeparpinelli/word2vec-pt-br